# Contexto ✒️

Continuação da estudo com o modelo BLIP, o qual é capaz de produzir texto a partir de imagens a ele fornecidas. Nessa etapa, irei integrar o modelo com uma interface por meio do Gradio, um pacote open source, que permite a criação de interfaces amigáveis para a apresentação da funcionalidade de modelos de machine learnig de forma rápida e fácil.  

## Bibliotecas 📚

In [1]:
!pip install transformers -q

!pip install gradio -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.0 MB/s

In [9]:
import torch
import requests
import gradio as gr

from torchvision import transforms
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image


## Instanciando o processador e o modelo

O processador é responsável por transformar a imagem em um conjunto de tensores por meio dos quais a rede neural pode analisar a imagem e, a partir deles, extrair significado materializado em forma de texto. Como o computador não trabalha com linguagem natural, mas números, no primeiro momento da extração ele *escreve* as palavras em forma de vetores, do tipo tensor, as quais posteriormente são decodificadas para linguagem natural.

In [3]:
# Processador que traduz a imagem para a forma de tensor

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")

# Modelo que a partir dos tensores gera significado, através da compreensão da imagem.

model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [4]:
# Com o objetivo de utilizar o Gradio com o BLIP, irei criar uma função
# que a partir da imagem fornecida retona uma legenda.

# Para isso, irei utilizar de boas práticas, de modo
# a criar uma outra função que propriamente irá fazer
# a geração da legenda, enquanto outra irá apenas a executar.

def generate_caption(image):


  inputs = processor(image, return_tensors="pt")
  out = model.generate(**inputs)
  caption = processor.decode(out[0], skip_special_tokens=True)

  return caption

def caption_image(image):

  try:
    caption = generate_caption(image)
    return caption

  except Exception as e:
    return f"An error occurred : {str(e)}"

In [5]:
interface_gradio = gr.Interface(
    fn=caption_image,
    inputs=gr.Image(type="pil"),
    outputs="text",
    title="Image Captioning with BLIP",
    description="Upload an image and get a caption!"
)

In [6]:
# Para acessar a interface, basta descomentar a linha abaixo.

# interface_gradio.launch()

## Classificador de imagens

Vimos a implementação do BLIP com o Gradio, assim como nesse repositório já tivemos contado com o modelo BLIP e compreendemos o seu funcionamento, o qual é separado na etapa de processamento da imagem, transformando-a em tensor e depois em sua capitalização, gerando legendas.

Porém, como ocorre a classificação de um modelo de imagens ? Como ele considera um determinado objeto correspondente e acerta ou erra ? Em ordem de satisfazer essas perguntas, irei trazer uma rede neural que classifica imagens, aplicando-a junto com o Gradio, para facilitar a compreensão de seu funcionamento.

In [8]:
torch_model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True).eval()

Downloading: "https://github.com/pytorch/vision/zipball/v0.10.0" to /root/.cache/torch/hub/v0.10.0.zip
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 105MB/s]


In [10]:
# Download human-readable labels for ImageNet.
response = requests.get("https://git.io/JJkYN")
labels = response.text.split("\n")

def predict(inp):

  # Transforma a entrada em um tensor, adicionando no processo
  # mais uma dimensão por necessidade da rede neural de uso.

 inp = transforms.ToTensor()(inp).unsqueeze(0)

 # Esse trecho faz com que os cálculos circunscritos a ele não
 # sejam rastreados para fins de otimização de processamento,
 # exigindo menos espaço na memória.
 with torch.no_grad():

  # A imagem é passada para a rede neural a qual realiza uma previsão
  # em meio a probabilidade de cada rótulo, com um determinado intervalo de confiança.

  prediction = torch.nn.functional.softmax(model(inp)[0], dim=0)
  confidences = {labels[i]: float(prediction[i]) for i in range(1000)}

  return confidences

In [11]:
# Criando a interface no Gradio para o uso do modelo que classifica imagens.

gr.Interface(fn=predict,
       inputs=gr.Image(type="pil"),
       outputs=gr.Label(num_top_classes=3),
       examples=["/content/lion.jpg", "/content/cheetah.jpg"]).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://f73adad957cf066f41.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [12]:
gr.close_all()

Closing server running on port: 7860
